## Import the required libraries


In [1]:
import timeit

import numpy as np

import pyvinecopulib as pv

from collections import defaultdict

## Set the parameters for the benchmark

In [2]:
# TODO: Check TLL because compiler might have large impact on performance there

n = 1000
d = 5
repeats = 10


def generate_data(n, d, seed=0):
  np.random.seed(seed)
  # Generate data
  x = np.random.normal(size=n).reshape(n, 1) * np.ones(
    (n, d)
  ) + 0.5 * np.random.normal(size=(n, d))

  # Convert data to pseudo-observations
  u = pv.to_pseudo_obs(x)

  return u

## Define the functions to be used in the benchmark

In [3]:
controls_itau = pv.FitControlsVinecop(family_set=pv.itau, num_threads=1)
controls_itau_par_method = pv.FitControlsVinecop(
  family_set=pv.itau, parametric_method="itau", num_threads=1
)
controls_tll = pv.FitControlsVinecop(family_set=[pv.tll], num_threads=1)


# Define two different configurations of the `vinecop` function
def vinecop_itau(u):
  return pv.Vinecop.from_data(u, controls=controls_itau)


def vinecop_itau_par_method(u):
  return pv.Vinecop.from_data(u, controls=controls_itau_par_method)


def vinecop_tll(u):
  return pv.Vinecop.from_data(u, controls=controls_tll)


functions_to_benchmark = {
  "itau": vinecop_itau,
  "itau_par_method": vinecop_itau_par_method,
  "tll": vinecop_tll,
}

## Run the benchmark

In [4]:
# Benchmark the functions
results = defaultdict(list)
for seed in range(repeats):
  u = generate_data(n, d, seed)
  for name, function in functions_to_benchmark.items():
    tic = timeit.default_timer()
    function(u)
    toc = timeit.default_timer()
    results[name].append(toc - tic)


def print_results(results, name):
  print(
    f"{name}: min={min(results)*1000:.6f}, mean={np.mean(results)*1000:.6f}, max={max(results)*1000:.6f}"
  )


# Display benchmark results
print("Unit: milliseconds:")
for name, result in results.items():
  print_results(result, name)

Unit: milliseconds:
itau: min=1371.399782, mean=1560.420646, max=1805.472184
itau_par_method: min=178.614352, mean=184.697386, max=191.615308
tll: min=379.891949, mean=388.168963, max=398.945237
